<a href="https://colab.research.google.com/github/rapsoj/crop-yield-estimate/blob/main/01.02-feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03.01 XGBoost
Building the XGBoost model for the Digital Green Crop Yield Estimate Challenge.

### Prepare Workspace

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Import data manipulation libraries
import pandas as pd
import numpy as np

In [ ]:
# Load files
data_path = '/content/drive/MyDrive/Colab Notebooks/crop-yield-estimate/'
train = pd.read_csv(data_path + 'Train.csv')
test = pd.read_csv(data_path + 'Test.csv')
var_desc = pd.read_csv(data_path + 'VariableDescription.csv')

### Prepare Workspace Locally

In [2]:
import pandas as pd
import numpy as np

In [14]:
data_path = '/crop-yield-estimate/data/'
train = pd.read_csv(data_path + 'Train.csv')
test = pd.read_csv(data_path + 'Test.csv')
var_desc = pd.read_csv(data_path + 'VariableDescription.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/crop-yield-estimate/data/Train.csv'

In [ ]:
train.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,...,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
0,ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,45,40,TractorPlough FourWheelTracRotavator,2022-07-20,5,Manual_PuddledRandom,2022-06-27,...,machine,2022-11-16,NaN,2022-11-16,machine,30,40,plowed_in_soil,0.312500,600
1,ID_TK40ARLSPOKS,Nalanda,Rajgir,26,26,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,5,Manual_PuddledRandom,2022-06-20,...,hand,2022-11-25,3.0,2022-12-24,machine,24,10,plowed_in_soil,0.312500,600
2,ID_1FJY2CRIMLZZ,Gaya,Gurua,10,10,TractorPlough FourWheelTracRotavator,2022-06-30,6,Manual_PuddledRandom,2022-06-20,...,hand,2022-12-12,480.0,2023-01-11,machine,30,10,plowed_in_soil,0.148148,225
3,ID_I3IPXS4DB7NE,Gaya,Gurua,15,15,TractorPlough FourWheelTracRotavator,2022-06-16,6,Manual_PuddledRandom,2022-06-17,...,hand,2022-12-02,240.0,2022-12-29,hand,26,10,plowed_in_soil,0.222222,468
4,ID_4T8YQWXWHB4A,Nalanda,Noorsarai,60,60,TractorPlough WetTillagePuddling,2022-07-19,4,Manual_PuddledRandom,2022-06-21,...,machine,2022-11-30,NaN,2022-12-02,machine,24,40,plowed_in_soil,0.468750,550


In [ ]:
count = len(train[train['CultLand'] < train['CropCultLand']])
print(count)


0


In [ ]:
count = len(train[train['CultLand'] > train['CropCultLand']])
print(count)


1616


In [4]:

# Calculate the correlation between CultLand and Acre columns
corr_cultland = train['CultLand'].corr(train['Acre'])

# Calculate the correlation between CropCultLand and Acre columns
corr_cropcultland = train['CropCultLand'].corr(train['Acre'])

print(corr_cultland)
print(corr_cropcultland)



NameError: name 'train' is not defined

In [5]:
# How many missing numbers are there?
train.isnull().sum()

NameError: name 'train' is not defined

In [ ]:
# Calculate percentage of missing values in each column
train.isnull().sum() / len(train) * 100

ID                                     0.000000
District                               0.000000
Block                                  0.000000
CultLand                               0.000000
CropCultLand                           0.000000
LandPreparationMethod                  0.000000
CropTillageDate                        0.000000
CropTillageDepth                       0.000000
CropEstMethod                          0.000000
RcNursEstDate                          2.144703
SeedingSowingTransplanting             0.000000
SeedlingsPerPit                        7.467700
NursDetFactor                          7.467700
TransDetFactor                         7.467700
TransplantingIrrigationHours           4.987080
TransplantingIrrigationSource          2.971576
TransplantingIrrigationPowerSource    12.997416
TransIrriCost                         22.790698
StandingWater                          6.149871
OrgFertilizers                        34.496124
Ganaura                               62

### Perform Feature Engineering

In [6]:
# Create feature for yield per acre 'Yield_per_Acre'
train['Yield_per_Acre'] = train['Yield'] / train['Acre']

NameError: name 'train' is not defined

In [ ]:
# Create feature for past month yield per acre 'Past_Yield_per_Acre'
train['Harv_date'] = pd.to_datetime(train['Harv_date'])
train.sort_values(['District', 'Harv_date'], inplace=True)

# Group the DataFrame by 'District' and calculate the rolling average
train['Past_YpA_Avg'] = train.groupby('District')['Yield_per_Acre'].rolling(
    window = 30).mean().reset_index(0, drop=True)

# Fill NaN values in the 'past_month_avg' column with 0 if needed
train['Past_YpA_Avg'].fillna(0, inplace=True)

In [7]:
# Create feature for days between harvesting and threshing 'Days_Harv_Thresh'
train['Threshing_date'] = pd.to_datetime(train['Threshing_date'])
train['Days_Harv_Thresh'] = (
    train['Threshing_date'] - train['Harv_date']).dt.days

NameError: name 'train' is not defined

# XGBoost Model - Shaw - Just testing the model


In [8]:
import pandas as pd
import numpy as np
import xgboost as xgb

# If you run the feature engineering workbook, it'll genereate the file 
data = pd.read_csv("/crop-yield-estimate/02-data-cleaning/cleaned_fulldf_train.csv")

# Obv depends how data is formatted
X = data.drop(['Yield', 'Yield_per_Acre'], axis=1)
y = data['Yield']


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# lets just double check that was done right
X

,ID,Set,District,Block,CultLand,CropTillageDate,CropTillageMonth,CropTillageDepth,CropEstMethod,RcNursEstDate,...,Days_bw_SowTransp_Harv,Days_bw_Harv_Thresh,Days_bw_Nurs_Harv,Nb_of_NaN,Past_YpA_Avg,Days_Harv_Thresh,Season,harvest-thresh-duration,tillage-harvest-duration,irrigation-per-acre
0,ID_OEGIGBAY4B8L,train,Gaya,Wazirganj,30,2022-06-28,June,4,Manual_PuddledRandom,2022-06-20,...,77.0,14.0,110.0,1,0.000000,14,Monsoon,116,102,16.20
1,ID_ECOLA7UOIRKC,train,Gaya,Wazirganj,50,2022-07-01,July,5,Manual_PuddledRandom,2022-06-25,...,74.0,17.0,107.0,4,0.000000,17,Monsoon,118,101,11.25
2,ID_T882H8WXTQWU,train,Gaya,Wazirganj,40,2022-06-28,June,5,Manual_PuddledRandom,2022-06-28,...,76.0,18.0,104.0,1,0.000000,18,Monsoon,122,104,27.00
3,ID_5Q6AC5G0B74I,train,Gaya,Wazirganj,40,2022-06-28,June,5,Manual_PuddledRandom,2022-06-26,...,76.0,18.0,108.0,2,0.000000,18,Monsoon,124,106,27.00
4,ID_RIJLIV1YFCB9,train,Gaya,Wazirganj,40,2022-07-07,July,5,Manual_PuddledRandom,2022-06-27,...,78.0,18.0,107.0,5,0.000000,18,Monsoon,115,97,10.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865,ID_6IR4GXDDI7BN,train,Vaishali,Mahua,4,2022-07-14,July,4,Manual_PuddledRandom,2022-07-16,...,104.0,7.0,130.0,0,2027.666667,7,Monsoon,139,132,22.00
3866,ID_XO7TC7OX6CUF,train,Vaishali,Mahua,5,2022-07-02,July,5,Manual_PuddledRandom,2022-07-04,...,116.0,0.0,142.0,1,2052.111111,0,Monsoon,144,144,22.00
3867,ID_PM8BF11ZSDTS,train,Vaishali,Mahua,5,2022-07-17,July,5,Manual_PuddledRandom,2022-07-19,...,100.0,0.0,127.0,1,1997.111111,0,Monsoon,129,129,22.00
3868,ID_H9X51NB7FWLX,train,Vaishali,Mahua,5,2022-07-17,July,5,Manual_PuddledRandom,2022-07-19,...,103.0,7.0,127.0,1,2009.333333,7,Monsoon,136,129,22.00


In [9]:
y

NameError: name 'y' is not defined

In [ ]:
# Convert all object columns to categorical
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')


Below code blocks taken from Data Camp

In [10]:
data_Dmatrix = xgb.DMatrix(data=X, label=y, enable_categorical=True)


NameError: name 'xgb' is not defined

In [ ]:
untuned_params = {"objective": "reg:squarederror"}


untuned_cv_results_rmse = xgb.cv(dtrain=data_Dmatrix,  params=untuned_params, nfold=4, metrics="rmse", as_pandas=True, seed=123)

print(untuned_cv_results_rmse["test-rmse-mean"].tail(1))

9    565.572765
Name: test-rmse-mean, dtype: float64


In [11]:
tuned_params = {"objective":"reg:squarederror", 'colsample_bytree':0.3, 'learning_rate': 0.1, 'max_depth':5}

tuned_cv_results_rmse = xgb.cv(dtrain=data_Dmatrix, params=tuned_params, nfold=4, num_boost_round=200, metrics="rmse", as_pandas=True, seed=123)

print("Tuned rmse: %f" %((tuned_cv_results_rmse["test-rmse-mean"]).tail(1)))

NameError: name 'xgb' is not defined

In [12]:

from sklearn.model_selection import GridSearchCV


gbm_param_grid ={
'learning_rate': [0.01, 0.1, 0.5, 0.9],
'n_estimators': [200],
'subsample': [0.3, 0.5, 0.9],
'max_depth': [4, 5, 6, 7, 8]
}

gbm = xgb.XGBRegressor(enable_categorical=True)

grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, scoring='neg_mean_squared_error', cv=4, verbose=1)

grid_mse.fit(X,y)

print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

NameError: name 'xgb' is not defined